In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import os, sys
from six.moves import cPickle as pickle
from six.moves import range
from collections import defaultdict


# Model Packages importA
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

from CIFAR_10 import genTrainValidFolds


# Packages for plot:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline


# Get the current working directory
# cwd = os.getcwd()
# print (cwd)
# # mydir = os.path.abspath(os.path.join(cwd, ".."))
# print (mydir)
# sys.path.append(cwd)

# Important Functions

In [ ]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels=None, imageSize=32, numChannels=1, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape((-1,imageSize,imageSize,numChannels)) # To reshape the
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape((-1,imageSize,imageSize,numChannels)) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

def accuracy(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])


def confusionMatrix(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (pd.crosstab(np.argmax(labels, 1), np.argmax(predictions, 1), rownames=['True'], colnames=['Predicted'], margins=True))


# Linear Activation:

In [3]:
# We create chunk of Reusable components:
# Here we create a re-usable funtion to add linear networks layers

# pass params as {wMean=0.0, wStdev=0.1}
def linearActivation(x, numInputs, numOutputs, params, scope=None):
    """
    :param x:          Input Data
    :param numInputs:  number of inputs
    :param numOutputs: number of outputs
    :param params:     parameters for linear function Examples {wMean=0.0, wStdev=0.1, wSeed=6432, bSeed=928}
    :param scope:      Scope name (Layer1 or Layer2 depending on what your layer is)
    :return:           A computational graph of the processes of the linear activation
    """
    with tf.variable_scope(scope or "linear"):
        w = tf.get_variable(
                dtype='float32',
                shape=[numInputs, numOutputs],
                initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1),
                name='weight')
        
        b = tf.get_variable(
                dtype='float32',
                shape=[numOutputs],
                initializer=tf.constant_initializer(),
                name='bias')
        
        yPred = tf.matmul(x,w) + b
        
        return yPred